In [1]:
#  Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,54.19,81,50,7.34,scattered clouds
1,1,Lompoc,US,34.6391,-120.4579,58.78,96,100,10.36,mist
2,2,Vila Franca Do Campo,PT,37.7167,-25.4333,73.20,83,100,11.95,overcast clouds
3,3,Esperance,AU,-33.8667,121.9000,57.04,72,99,23.82,light rain
4,4,Viransehir,TR,37.2353,39.7631,96.15,10,0,13.94,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Kudat,MY,6.8837,116.8477,76.93,94,20,0.00,few clouds
7,7,Sao Francisco,BR,-15.9486,-44.8644,87.60,34,53,9.01,light rain
12,12,Hami,CN,42.8000,93.4500,80.29,11,14,8.66,few clouds
15,15,Rikitea,PF,-23.1203,-134.9692,75.52,72,100,11.23,overcast clouds
26,26,Cabo San Lucas,MX,22.8909,-109.9124,79.61,68,0,8.90,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
complete_preferred_cities_df = pd.DataFrame(preferred_cities_df.dropna())
complete_preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Kudat,MY,6.8837,116.8477,76.93,94,20,0.00,few clouds
7,7,Sao Francisco,BR,-15.9486,-44.8644,87.60,34,53,9.01,light rain
12,12,Hami,CN,42.8000,93.4500,80.29,11,14,8.66,few clouds
15,15,Rikitea,PF,-23.1203,-134.9692,75.52,72,100,11.23,overcast clouds
26,26,Cabo San Lucas,MX,22.8909,-109.9124,79.61,68,0,8.90,clear sky
...,...,...,...,...,...,...,...,...,...,...
549,549,Sambava,MG,-14.2667,50.1667,75.56,76,0,16.20,clear sky
554,554,Maningrida,AU,-12.0667,134.2667,76.21,85,86,8.16,overcast clouds
556,556,Zhangzhou,CN,24.5133,117.6556,77.88,96,100,0.81,overcast clouds
558,558,Eyl,SO,7.9803,49.8164,81.97,70,19,24.85,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = complete_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng
6,Kudat,MY,76.93,few clouds,6.8837,116.8477
7,Sao Francisco,BR,87.60,light rain,-15.9486,-44.8644
12,Hami,CN,80.29,few clouds,42.8000,93.4500
15,Rikitea,PF,75.52,overcast clouds,-23.1203,-134.9692
26,Cabo San Lucas,MX,79.61,clear sky,22.8909,-109.9124
...,...,...,...,...,...,...
549,Sambava,MG,75.56,clear sky,-14.2667,50.1667
554,Maningrida,AU,76.21,overcast clouds,-12.0667,134.2667
556,Zhangzhou,CN,77.88,overcast clouds,24.5133,117.6556
558,Eyl,SO,81.97,few clouds,7.9803,49.8164


In [8]:
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kudat,MY,76.93,few clouds,6.8837,116.8477,
7,Sao Francisco,BR,87.60,light rain,-15.9486,-44.8644,
12,Hami,CN,80.29,few clouds,42.8000,93.4500,
15,Rikitea,PF,75.52,overcast clouds,-23.1203,-134.9692,
26,Cabo San Lucas,MX,79.61,clear sky,22.8909,-109.9124,
30,Hilo,US,82.27,moderate rain,19.7297,-155.0900,
32,Banjar,ID,78.12,broken clouds,-8.1900,114.9675,
37,Hambantota,LK,80.94,few clouds,6.1241,81.1185,
50,Bethel,US,78.15,few clouds,41.3712,-73.4140,
56,Alyangula,AU,76.46,scattered clouds,-13.8483,136.4192,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
   
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kudat,MY,76.93,few clouds,6.8837,116.8477,Dream Garden Hotel
7,Sao Francisco,BR,87.60,light rain,-15.9486,-44.8644,Pousada Peixe Vivo
12,Hami,CN,80.29,few clouds,42.8000,93.4500,Laoyutang Resort
15,Rikitea,PF,75.52,overcast clouds,-23.1203,-134.9692,People ThankYou
26,Cabo San Lucas,MX,79.61,clear sky,22.8909,-109.9124,Vista Roof Restaurant


In [12]:
# 8a. Create the output File CSV
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [14]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
print(hotel_info)

['\n<dl>\n<dt>Hotel Name</dt><dd>Dream Garden Hotel</dd>\n<dt>City</dt><dd>Kudat</dd>\n<dt>Country</dt><dd>MY</dd>\n<dt>Current Description</dt><dd>few clouds and 76.93 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>Pousada Peixe Vivo</dd>\n<dt>City</dt><dd>Sao Francisco</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Description</dt><dd>light rain and 87.6 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>Laoyutang Resort</dd>\n<dt>City</dt><dd>Hami</dd>\n<dt>Country</dt><dd>CN</dd>\n<dt>Current Description</dt><dd>few clouds and 80.29 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Description</dt><dd>overcast clouds and 75.52 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>Vista Roof Restaurant</dd>\n<dt>City</dt><dd>Cabo San Lucas</dd>\n<dt>Country</dt><dd>MX</dd>\n<dt>Current Description</dt><dd>clear sky and 79.61 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>Hilo Hawaiian

In [15]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
6,6.8837,116.8477
7,-15.9486,-44.8644
12,42.8000,93.4500
15,-23.1203,-134.9692
26,22.8909,-109.9124
...,...,...
549,-14.2667,50.1667
554,-12.0667,134.2667
556,24.5133,117.6556
558,7.9803,49.8164


In [22]:
import matplotlib as plt

In [25]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))